# Detect and crop images
- using model pre-trained on COCO

In [1]:
import torchvision
import numpy
from PIL import Image
import cv2
import torchvision.transforms as T
import matplotlib.pyplot as plt
import os 
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=True)
          (downsample)

In [4]:
# label list => we are using 'dog' or 'cat'
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


In [6]:
# prediction + bounding box 좌표 찾기, 한국어 경로 지원
# 강아지 검출이 안됐을 시 error = 1 로 설정.
def get_prediction(img_path, threshold):
    stream = open( img_path.encode("utf-8") , "rb")
    bytes = bytearray(stream.read())
    numpyArray = numpy.asarray(bytes, dtype=numpy.uint8)
    img = cv2.imdecode(numpyArray , cv2.IMREAD_UNCHANGED)
    src = img.copy() 
    error = 0 
   # img = Image.open(img_path) # Load the image
    transform = T.Compose([T.ToTensor()]) # Defing PyTorch Transform
    img = transform(img) # Apply the transform to the image
    pred = model([img]) # Pass the image to the model
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())] # Get the Prediction Score
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())] # Bounding boxes
    pred_score = list(pred[0]['scores'].detach().numpy())
    try:
        pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] # Get list of index with score greater than threshold.
    except IndexError as e:
        print(e, ' 강아지 검출 x')
        error = 1
        pred_t = 0
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    return pred_boxes, pred_class, src, error


In [8]:
# bbox 좌표를 더 크게 잡아서 crop 함수로 전달 
def object_detection_api(img_path, threshold=0.8, rect_th=1, text_size=1, text_th=1):
    boxes, pred_cls, img, error = get_prediction(img_path, threshold) # Get predictions
    if error == 1:
        return (0,0),(0,0),0,error
    img2 = img
    for i in range(len(boxes)):
        if pred_cls[i] == 'dog':
            #cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th) # Draw Rectangle with the coordinates
            print(pred_cls)
            print(boxes[i][0],boxes[i][1])
            lt = boxes[i][0]
            ld = (boxes[i][0][0], boxes[i][1][1])
            rt = (boxes[i][1][0], boxes[i][0][1])
            rd = boxes[i][1]
#             print('left top = ' , lt)
#             print('left down = ' , ld)
#             print('right top = ' , rt)
#             print('right down = ' , rd)
            #cv2.putText(img,pred_cls[i], boxes[i][0],  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class
            if int(lt[0] - 20) < 0:
                broad_lt_0 = 0
            else:
                broad_lt_0 = int(lt[0] - 20)
            if int(lt[1] - 20) < 0:
                broad_lt_1 = 0
            else:
                broad_lt_1 = int(lt[1] - 20)
        
            boxes_broad_lt = (broad_lt_0 , broad_lt_1)
            boxes_broad_rd = (int(rd[0] + 20) , int(rd[1] + 20) )
    #plt.figure(figsize=(30,30)) # display the output image
    #plt.imshow(img)
    return boxes_broad_lt, boxes_broad_rd, img2, error


In [7]:
# 한국어 경로 지원되는 image write 함수
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)
        
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f) 
            return True 
        else:
            return False
    except Exception as e:
        print(e)
        return False

In [10]:
#  이미지를 size에 맞게 crop후 저장하는 함수 
def crop_save(img_path, dst_name, size= 240):
    lt, rd, src, error = object_detection_api(img_path, threshold=0.9)
    if error == 1:
        return
   # dst_name = img_path.split('/')[-1].split('.jpg')[0] +'_crop'+'.jpg'
    print(lt,rd)
    roi = src[lt[1]:rd[1], lt[0]:rd[0]]
#    plt.imshow(roi)
    print(dst_name)
    dst = cv2.resize(roi, dsize=(size, size), interpolation=cv2.INTER_AREA)
    imwrite(dst_name,dst)

In [9]:
# main code
# 대상 이미지 class가 모여있는 폴더를 path_file에 입력. 
path_file = './' + 'sample'

for count, foldername in enumerate(os.listdir(path_file)): 
    print(count, foldername)
    for count, filename in enumerate(os.listdir(path_file+ '/' +foldername)): 
        print(count, filename)
        dst_name = filename.split('/')[-1].split('.jpg')[0] +'_crop'+'.jpg'
        path_src_file = path_file+'/'+foldername + '/'+filename
        path_dst_file =  path_file +'/'+foldername + '/'+ dst_name
        print(path_src_file)
        crop_save(path_src_file, path_dst_file)

0 골든_리트리버
0 고골든_리트리버_경기-성남-2020-00499.jpg
./sample/골든_리트리버/고골든_리트리버_경기-성남-2020-00499.jpg
1 골든_리트리버_강원-춘천-2020-00366.jpg
./sample/골든_리트리버/골든_리트리버_강원-춘천-2020-00366.jpg
2 골든_리트리버_강원-춘천-2020-00366_crop.jpg
./sample/골든_리트리버/골든_리트리버_강원-춘천-2020-00366_crop.jpg
3 골든_리트리버_강원-춘천-2020-00371.jpg
./sample/골든_리트리버/골든_리트리버_강원-춘천-2020-00371.jpg
4 골든_리트리버_강원-춘천-2020-00371_crop.jpg
./sample/골든_리트리버/골든_리트리버_강원-춘천-2020-00371_crop.jpg
5 골든_리트리버_강원-횡성-2020-00114.jpg
./sample/골든_리트리버/골든_리트리버_강원-횡성-2020-00114.jpg
6 골든_리트리버_강원-횡성-2020-00114_crop.jpg
./sample/골든_리트리버/골든_리트리버_강원-횡성-2020-00114_crop.jpg
7 골든_리트리버_경기-가평-2020-00184.jpg
./sample/골든_리트리버/골든_리트리버_경기-가평-2020-00184.jpg
8 골든_리트리버_경기-가평-2020-00184_crop.jpg
./sample/골든_리트리버/골든_리트리버_경기-가평-2020-00184_crop.jpg
9 골든_리트리버_경기-고양-2020-00757.jpg
./sample/골든_리트리버/골든_리트리버_경기-고양-2020-00757.jpg
10 골든_리트리버_경기-고양-2020-00757_crop.jpg
./sample/골든_리트리버/골든_리트리버_경기-고양-2020-00757_crop.jpg
11 골든_리트리버_경기-부천-2020-00772.jpg
./sample/골든_리트리버/골든_리트리버_경기-부천-2020-00772.jpg
12 골든_리트리버_경

./sample/닥스훈트/닥스훈트_경기-의정부-2020-00271.jpg
10 닥스훈트_경기-의정부-2020-00284.jpg
./sample/닥스훈트/닥스훈트_경기-의정부-2020-00284.jpg
11 닥스훈트_경기-이천-2020-00533.jpg
./sample/닥스훈트/닥스훈트_경기-이천-2020-00533.jpg
12 닥스훈트_경기-평택-2020-01610.jpg
./sample/닥스훈트/닥스훈트_경기-평택-2020-01610.jpg
13 닥스훈트_경기-화성-2020-01285.jpg
./sample/닥스훈트/닥스훈트_경기-화성-2020-01285.jpg
14 닥스훈트_경기-화성-2020-01286.jpg
./sample/닥스훈트/닥스훈트_경기-화성-2020-01286.jpg
15 닥스훈트_경기-화성-2020-01292.jpg
./sample/닥스훈트/닥스훈트_경기-화성-2020-01292.jpg
16 닥스훈트_경기-화성-2020-01293.jpg
./sample/닥스훈트/닥스훈트_경기-화성-2020-01293.jpg
17 닥스훈트_경남-김해-2020-01139.jpg
./sample/닥스훈트/닥스훈트_경남-김해-2020-01139.jpg
18 닥스훈트_경남-양산-2020-00189.jpg
./sample/닥스훈트/닥스훈트_경남-양산-2020-00189.jpg
19 닥스훈트_경남-통영-2020-00271.jpg
./sample/닥스훈트/닥스훈트_경남-통영-2020-00271.jpg
20 닥스훈트_경남-통영-2020-00284.jpg
./sample/닥스훈트/닥스훈트_경남-통영-2020-00284.jpg
21 닥스훈트_경남-하동-2020-00205.jpg
./sample/닥스훈트/닥스훈트_경남-하동-2020-00205.jpg
22 닥스훈트_경북-성주-2020-00275.jpg
./sample/닥스훈트/닥스훈트_경북-성주-2020-00275.jpg
23 닥스훈트_경북-영주-2020-00252.jpg
./sample/닥스훈트/닥스훈트_경북-영주-2020-0